In [1]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [2]:
data = {
    "keywords": "anies",
    "since_time": "2023-09-01",
    "until_time": "2023-11-10"
}

In [3]:
url = f"https://indeks.kompas.com/?site=news&date=2023-09-01&page=100"  
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")
articles = soup.find_all('h3', {"class": "article__title article__title--medium"})
print(articles)

[]


In [4]:
def scrape_links(date,page_number):
    date=datetime.strftime()
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://indeks.kompas.com/?site=nasional&date={date}&page={page_number}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('h3', {"class": "article__title article__title--medium"})
    links = []
    for article in articles:
        link = article.find('a')['href']
        links.append(link)
    print(f"Scraped {len(links)} links from page {page_number}")
    
    return links

In [5]:
def scrape_link_per_day(date, max_threads=5):
    page_number = 1
    page_links = []

    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        futures = []

        while True:
            future = executor.submit(scrape_links, date, page_number)
            futures.append(future)
            page_number += 1

            # Break the loop if no more articles are found
            if not future.result():
                break

        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [6]:
link=scrape_link_per_day(data["since_time"])

Scraped 15 links from page 1
Scraped 15 links from page 2
Scraped 15 links from page 3
Scraped 15 links from page 4
Scraped 15 links from page 5
Scraped 15 links from page 6
Scraped 15 links from page 7
Scraped 15 links from page 8
Scraped 15 links from page 9
Scraped 1 links from page 10
Scraped 0 links from page 11


In [7]:
print(link)

['https://nasional.kompas.com/read/2023/09/01/00000011/tanggal-3-september-memperingati-hari-apa-', 'https://nasional.kompas.com/read/2023/09/01/06120051/pakar-sarankan-pemerintah-duduk-bareng-swasta-samakan-indeks-pengukur', 'https://nasional.kompas.com/read/2023/09/01/06030331/ahy-ayo-segera-move-on-cari-pacar-lagi', 'https://nasional.kompas.com/read/2023/09/01/06020061/muncul-isu-anies-muhaimin-ppp-terbuka-jika-demokrat-atau-pks-pindah-ke', 'https://nasional.kompas.com/read/2023/09/01/06010031/surya-paloh-harap-koalisi-pengusung-anies-tidak-bubar-usai-demokrat-tuding', 'https://nasional.kompas.com/read/2023/09/01/05514171/surya-paloh-sebut-keputusan-resmi-anies-muhaimin-tunggu-perkembangan-politik', 'https://nasional.kompas.com/read/2023/09/01/05510051/bahas-pemilu-damai-2024-mendagri-kalau-kalah-jangan-marah-marah', 'https://nasional.kompas.com/read/2023/09/01/05455421/ada-orang-yang-mengaku-jubir-kpk-dan-hubungi-saksi-lukas-enembe', 'https://nasional.kompas.com/read/2023/09/01/054

In [12]:
def scrape_url(url,max_retries=2):
    retries = 0
    while retries < max_retries:
            try:
                headers = {
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
                }
                response = requests.get(url, headers=headers)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    
                    
                    # Judul Berita
                    title_elem = soup.find('h1', {"class": "read__title"})
                    if title_elem:
                        title_text = title_elem.text.strip()
                    else:
                        title_text = "Title not found"  
                    # tanggal berita
                    date_elem = soup.find('div', {"class": "read__time"})
                    if date_elem:
                        date_text = date_elem.text.strip()
                        date_part = date_text.split(' - ')[1].split(',')[0].strip()
                        date_object = datetime.strptime(date_part, '%d/%m/%Y')
                        formatted_date = date_object.strftime('%Y/%m/%d')
                    else:
                        date_text = "Date not found"
                    #     # Content Berita
                    body_elem = soup.find('div', {"class": "read__content"})
                        
                    if body_elem:
                        content_elem = body_elem.find_all('p')
                        content_text = ""
                        for p in content_elem:
                                content_text += p.text.strip() + "\n"
                            
                        if content_text.strip():
                            content_text=content_text
                            content_text = content_text.replace('\n', '').replace('\r', '').replace('\t', '')
                            content_text = ' '.join(content_text.split())
                        else:
                            content_text="Content not found"
                    else:
                            content_text="Content not found"

                    return{
                        'title': title_text,
                        'date': formatted_date,
                        'content':content_text,
                        'link' : url}
                elif response.status_code == 429:
                    print(f"Received a 429 error for {url}. Retrying in 5 seconds...")
                    time.sleep(5)
                else:
                    print(f"Failed to retrieve data from {url}: Status Code {response.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Error fetching URL '{url}': {e}")
            except Exception as e:
                print(f"Error processing URL '{url}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying {url} (Attempt {retries}/{max_retries})")
                time.sleep(5)  # You can adjust the delay as needed
    return None         

In [13]:
url='https://nasional.kompas.com/read/2023/09/01/06030331/ahy-ayo-segera-move-on-cari-pacar-lagi'
data_kompas = scrape_url(url)

In [14]:
print(data_kompas)

{'title': 'AHY, Ayo Segera "Move On" Cari "Pacar" Lagi', 'date': '2023/09/01', 'content': '"Hampa itu seperti langkah tak berjejak, senja tapi tak jingga, cinta tapi tak dianggap."KALIMAT galau yang saya kerap dengar dari kisah remaja saat putus cinta ternyata tidak melulu terjadi pada percintaan para remaja, tetapi juga tengah melanda partai-partai politik terutama kepada calon-calon pemimpin negeri.Saat Jakarta dan beberapa kota di negeri ini sedang “tidak baik-baik saja” karena polusi yang mengepung di sana-sini, jagat politik sejak Kamis petang (31 Agustus 2023) langsung menghangat usai surat dari Partai Demokrat yang berisi pernyataan Sekjen Teuku Riefky Harsya menyebar di berbagai lini masa.Sekjen Partai Demokrat Teuku Riefky Harsya yang juga anggota Tim 8 bentukan Demokrat, Nasdem dan PKS yang membantu bakal capres Anies Baswedan dalam merumuskan strategi pemenangan termasuk menggodok isu-isu strategis Koalisi Perubahan mengeluarkan pernyataan yang “menghentak”.Demokrat merasa d

In [9]:
# for url in link:
#     data_kompas = scrape_url(url)
#     print(data["keywords"])
#     keywords=data["keywords"]
#     if keywords.lower() in data_kompas['title'].lower() or keywords.lower() in data_kompas['content'].lower():
#         print("Data contains keywords:", data_kompas)
#     else:
#         print("News does not contain the specified keywords and will not be inserted into the database. URL:", data_kompas['link'])

anies
News does not contain the specified keywords and will not be inserted into the database. URL: https://nasional.kompas.com/read/2023/09/01/00000011/tanggal-3-september-memperingati-hari-apa-
anies
News does not contain the specified keywords and will not be inserted into the database. URL: https://nasional.kompas.com/read/2023/09/01/06120051/pakar-sarankan-pemerintah-duduk-bareng-swasta-samakan-indeks-pengukur
anies
Data contains keywords: {'title': 'AHY, Ayo Segera "Move On" Cari "Pacar" Lagi', 'date': 'Kompas.com - 01/09/2023, 06:03 WIB', 'content': '"Hampa itu seperti langkah tak berjejak, senja tapi tak jingga, cinta tapi tak dianggap."KALIMAT galau yang saya kerap dengar dari kisah remaja saat putus cinta ternyata tidak melulu terjadi pada percintaan para remaja, tetapi juga tengah melanda partai-partai politik terutama kepada calon-calon pemimpin negeri.Saat Jakarta dan beberapa kota di negeri ini sedang “tidak baik-baik saja” karena polusi yang mengepung di sana-sini, jagat

KeyboardInterrupt: 